# Convert openFEMA .csv files to spatial format 

#### Import modules:

In [1]:
# Pandas to handle all of the .csv files downloaded from openFEMA.
import pandas as pd
# Glob will be used to aggregate file lists from folders.
import glob
# OS will be used for paths.
import os

#### Preparation:

In [2]:
# These are the columns that exist in the first policy .csv, which is broken into 11 parts.
#  This list is used to assign headers to subsequent policy .csv files which do not have headers. 
policy_columns = [
    'agriculturestructureindicator', 'basefloodelevation',
    'basementenclosurecrawlspacetype', 'cancellationdateoffloodpolicy',
    'censustract', 'condominiumindicator', 'construction', 'countycode',
    'crsdiscount', 'deductibleamountinbuildingcoverage',
    'deductibleamountincontentscoverage', 'elevatedbuildingindicator',
    'elevationcertificateindicator', 'elevationdifference',
    'federalpolicyfee', 'floodzone', 'hfiaasurcharge',
    'houseofworshipindicator', 'latitude', 'locationofcontents',
    'longitude', 'lowestadjacentgrade', 'lowestfloorelevation',
    'nonprofitindicator', 'numberoffloorsininsuredbuilding',
    'obstructiontype', 'occupancytype', 'originalconstructiondate',
    'originalnbdate', 'policycost', 'policycount', 'policyeffectivedate',
    'policyterminationdate', 'policytermindicator',
    'postfirmconstructionindicator', 'primaryresidenceindicator',
    'propertystate', 'reportedzipcode', 'ratemethod',
    'regularemergencyprogramindicator', 'reportedcity',
    'smallbusinessindicatorbuilding', 'totalbuildinginsurancecoverage',
    'totalcontentsinsurancecoverage', 'totalinsurancepremiumofthepolicy'
]
# These are the columns that we want to keep for each policy .csv file.
policy_use_columns = [
    'countycode',
    'censustract',
    'latitude',
    'longitude',
    'totalbuildinginsurancecoverage',
    'policyeffectivedate']

# These are the relative paths for each openFEMA policy .csv file.
policy_paths = [
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_1/openFEMA_policies20190831_01.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_1/openFEMA_policies20190831_02.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_2/openFEMA_policies20190831_03.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_2/openFEMA_policies20190831_04.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_3/openFEMA_policies20190831_05.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_3/openFEMA_policies20190831_06.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_4/openFEMA_policies20190831_07.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_4/openFEMA_policies20190831_08.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_5/openFEMA_policies20190831_09.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_5/openFEMA_policies20190831_10.csv',
    'Data/FIMA_NFIP_Redacted_Policies_Data_Set_Part_6/openFEMA_policies20190831_11.csv'
]

#### Pull Indiana data from policy .csv files and write info to a new file:

In [3]:
for index, path in enumerate(policy_paths):
    # If it is the first .csv which has headers:
    if index == 0:
        # Store the correct policy number based on file name.
        policy_number = index + 1
        
        # Import policy info as DataFrame with only the applicable columns.
        openFEMA_policy_df = pd.read_csv(path, usecols=policy_use_columns)

        # Sort out all rows that are not from Indiana ('countycode' must start with 18).
        indiana_policies = openFEMA_policy_df[openFEMA_policy_df['countycode'].apply(str).str.startswith('18') == True]
        
        # Drop rows that are missing values, which will interfere with later operations.
        indiana_policies = indiana_policies.dropna()
        
        # Save the filtered DataFrame to a new file and delete DataFrames from memory.
        indiana_policies.to_csv('Data/Indiana_data/Indiana_Policies_{}.csv'.format(str(policy_number)))
        del openFEMA_policy_df, indiana_policies
    
    # For every other policy .csv file:
    else:
        # Store the correct policy number based on file name.
        policy_number = index + 1
        
        # Import policy info as DataFrame.
        openFEMA_policy_df = pd.read_csv(path, low_memory=False)
        
        # Assign names to columns in DataFrame.
        openFEMA_policy_df.columns = policy_columns
        
        # Take only the columns we need.
        openFEMA_policy_df = openFEMA_policy_df[policy_use_columns].copy()
        
        # Sort out all rows that are not from Indiana ('countycode' must start with 18).
        indiana_policies = openFEMA_policy_df[openFEMA_policy_df['countycode'].apply(str).str.startswith('18') == True]
        
        # Drop rows that are missing values, which will interfere with later operations.
        indiana_policies = indiana_policies.dropna()
        
        # Save the filtered DataFrame to a new file and delete DataFrames from memory.
        indiana_policies.to_csv('Data/Indiana_data/Indiana_Policies_{}.csv'.format(str(policy_number)))
        del openFEMA_policy_df, indiana_policies

#### Convert Indiana policy .csv files to point feature classes and merge together:

In [4]:
# Set workspace for all future operations.
arcpy.env.workspace = 'openFEMA_to_HUC12.gdb'

# Collect the policy .csv files as a list of file paths.
indiana_data_directory = os.path.relpath('Data/Indiana_data')
indiana_policy_csv_list = glob.glob(indiana_data_directory + '\\Indiana_Policies_*')

# Convert the .csv files to spatial data using the latitude longitude fields.
#  Names of created feature classes correspond with names of .csv files.
for csv_path in indiana_policy_csv_list:
    fc_name = csv_path[18:-4]
    arcpy.management.XYTableToPoint(
        csv_path,
        fc_name,
        x_field='longitude',
        y_field='latitude'
    )

# Gather the policy feature classes as a list and merge them together.
policy_fc_list = arcpy.ListFeatureClasses('Indiana_Policies_*')
arcpy.Merge_management(policy_fc_list, 'Indiana_Policies_Merged')

<Result 'Indiana Sensitivity Analysis.gdb\\Indiana_Policies_Merged'>

#### Clean GeoDatabase of intermediate files:

In [5]:
# Clean GeoDatabase of intermediate files:
for num in range(1,12):
    fc_name = "Indiana_Policies_{}".format(num)
    arcpy.Delete_management(fc_name)

#### Pull Indiana data from claims .csv file, write to new file and convert to point feature class:

In [6]:
# Perform the same operations on the single 'Claims' .csv file.
claims_path = 'Data/FIMA_NFIP_Redacted_Claims_Data_Set/openFEMA_claims20190831.csv'

claims_use_columns = ['countycode', 'censustract', 'latitude', 'longitude', 'amountpaidonbuildingclaim', 'yearofloss']

# Import claims info as DataFrame with only the applicable columns.
openFEMA_claims_df = pd.read_csv(claims_path, usecols=claims_use_columns)

# Sort out all rows that are not from Indiana ('countycode' must start with 18)
indiana_claims = openFEMA_claims_df[openFEMA_claims_df['countycode'].apply(str).str.startswith('18') == True]

# Drop rows that are missing values
indiana_claims = indiana_claims.dropna()

# Save to new file and delete dataframes from memory
indiana_claims_csv = 'Data/Indiana_data/Indiana_Claims.csv'
indiana_claims.to_csv(indiana_claims_csv)
del openFEMA_claims_df, indiana_claims

# Convert the csv to spatial data using the lat lon fields
arcpy.management.XYTableToPoint(
    indiana_claims_csv,
    'Indiana_Claims',
    x_field='longitude',
    y_field='latitude'
)

<Result 'Indiana Sensitivity Analysis.gdb\\Indiana_Claims'>

#### Summarize policies and claims data based on census tract:

In [7]:
# Compute statistics for policies find average insurance coverage values within each census tract
arcpy.analysis.Statistics(
    "Indiana_Policies_Merged",
    "Indiana_Policies_Statistics",
    "totalbuildinginsurancecoverage MEAN",
    "censustract"
)

<Result 'Indiana Sensitivity Analysis.gdb\\Indiana_Policies_Statistics'>

In [8]:
# Compute statistics for claims to find average claim payment values within each census tract
arcpy.analysis.Statistics(
    "Indiana_Claims",
    "Indiana_Claims_Statistics",
    "amountpaidonbuildingclaim MEAN",
    "censustract"
)

<Result 'Indiana Sensitivity Analysis.gdb\\Indiana_Claims_Statistics'>

#### Format fields in the policies and claims files and join statistic fields to the census tracts feature class:

In [9]:
# Create GEOID fields as text in Indiana_Policies_Statistics & Indiana_Claims_Statistics to join to GEOID text field in IN_Census_Tracts
arcpy.management.CalculateField(
    "Indiana_Claims_Statistics",
    "GEOID",
    "$feature.censustract",
    "ARCADE",
    field_type="TEXT"
)

arcpy.management.CalculateField(
    "Indiana_Policies_Statistics",
    "GEOID",
    "$feature.censustract",
    "ARCADE",
    field_type="TEXT"
)

# Join claims and policy values for each tract from statistics tables to the spatial data
arcpy.management.JoinField(
    "Indiana_Census_Tracts",
    "GEOID",
    "Indiana_Policies_Statistics",
    "GEOID",
    "MEAN_totalbuildinginsurancecoverage"
)

arcpy.management.JoinField(
    "Indiana_Census_Tracts",
    "GEOID",
    "Indiana_Claims_Statistics",
    "GEOID",
    "MEAN_amountpaidonbuildingclaim"
)

<Result 'Indiana_Census_Tracts'>

#### Clip HUC12 data to the state of Indiana and then calculate the area of each HUC12 in square kilometers:

In [10]:
# Clip HUC12 to state of Indiana to eliminate HUC12 area outside of state boundaries
arcpy.env.workspace = 'openFEMA_to_HUC12.gdb'
arcpy.analysis.Clip(
    "HUC12",
    "Indiana_State",
    "HUC12_Indiana"
)

<Result 'Indiana Sensitivity Analysis.gdb\\HUC12_Indiana'>

In [11]:
# Add field to store area in sq km 
arcpy.AddField_management(
    "HUC12_Indiana",
    "Area_sqkm",
    "DOUBLE"
)

# Calculate area of each clipped HUC12 in sqaure kilometers.
arcpy.management.CalculateGeometryAttributes(
    "HUC12_Indiana", 
    [["Area_sqkm", "AREA_GEODESIC"]],
    area_unit="SQUARE_KILOMETERS"
)

<Result 'HUC12_Indiana'>

#### Compute geometric union of clipped HUC12s and census tracts:

In [12]:
# Compute union of HUC12_Clip and IN_Census_Tracts
arcpy.env.workspace = 'openFEMA_to_HUC12.gdb'
arcpy.analysis.Union(
    ["HUC12_Indiana", "Indiana_Census_Tracts"],
   "HUC12_Tracts_Union",
)

<Result 'Indiana Sensitivity Analysis.gdb\\HUC12_Tracts_Union'>

#### Assign claims and policies data to HUC12 based on proportion of HUC12 covered by census tract:

In [13]:
# Add field to store area in sq km 
arcpy.AddField_management(
    "HUC12_Tracts_Union",
    "Union_area",
    "DOUBLE"
)

# Calculate geometry of each union feature.
arcpy.management.CalculateGeometryAttributes(
    "HUC12_Tracts_Union",
    [["Union_area", "AREA_GEODESIC"]],
    area_unit="SQUARE_KILOMETERS"
)

# Calculate the proportions of each tract feature vs the HUC12 it resides in as a new field
arcpy.management.CalculateField(
    "HUC12_Tracts_Union",
    "Huc12_proportion",
    "!Union_area! / !Area_sqkm!",
    "PYTHON3",
    field_type="Double"
)

<Result 'HUC12_Tracts_Union'>

In [14]:
# Calculate building coverage and claims values for each HUC12 using summary statistics (each is prop of 100% of huc coverage,
# sum together to get correct representative mean values for each huc)
arcpy.analysis.Statistics(
    "HUC12_Tracts_Union",
    "HUC12_Tracts_Union_Sums",
    "MEAN_totalbuildinginsurancecoverage SUM;MEAN_amountpaidonbuildingclaim SUM",
    "HUC12"
)

<Result 'Indiana Sensitivity Analysis.gdb\\HUC12_Tracts_Union_Sums'>

#### Create final layer for analysis:

In [15]:
# Join the summed averages back to the HUC12 file for analysis
arcpy.management.JoinField(
    "HUC12_Indiana",
    "HUC12",
    "HUC12_Tracts_Union_Sums",
    "HUC12",
    "SUM_MEAN_totalbuildinginsurancecoverage;SUM_MEAN_amountpaidonbuildingclaim"
)

<Result 'HUC12_Indiana'>